**Import required libraries**

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from random import shuffle
import math
import random
import matplotlib.pyplot as plt
import math
import seaborn as sns
import pickle
from datetime import datetime
import pandas as pd

In [34]:
class Connect2Model(nn.Module):

    def __init__(self, board_size, action_size, device):

        super(Connect2Model, self).__init__()

        self.device = device
        self.size = board_size
        self.action_size = action_size

        self.fc1 = nn.Linear(in_features=self.size, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=64)

        # Two heads on our network
        self.action_head = nn.Linear(in_features=64, out_features=self.action_size)
        self.value_head = nn.Linear(in_features=64, out_features=1)

        self.to(device)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        action_logits = self.action_head(x)
        value_logit = self.value_head(x)

        return F.softmax(action_logits, dim=1), value_logit

    def predict(self, board):
        board = torch.FloatTensor(board.astype(np.float32)).to(self.device)
        board = board.view(1, self.size)
        self.eval()
        with torch.no_grad():
            pi, v = self.forward(board)

        return pi.data.cpu().numpy()[0], v.data.cpu().numpy()[0]

SyntaxError: invalid syntax (221577311.py, line 5)

In [ ]:
import os
import numpy as np
from random import shuffle

import torch
import torch.optim as optim

class Trainer:

    def __init__(self, dl_model , model, args, eps):
        self.dl_model = dl_model
        self.model = model
        self.args = args
        self.mcts = MCTS_DL(self.dl_model, self.model, self.args)
        self.scores_mcts_dl = []
        self.scores_greedy = []
        self.scores_random = []
        self.scores_mcts = []

    def exceute_episode(self, tmp, plotting, random, greedy):

        train_examples = []
        state = []

        while True:

            self.mcts = MCTS_DL(self.dl_model, self.model, self.args)
            root = self.mcts.run(state)

            action_probs = [0 for _ in range(self.model.n_songs)]
            for k, v in root.children.items():
                action_probs[k] = v.visit_count

            action_probs = action_probs / np.sum(action_probs)
            train_examples.append((state, action_probs))

            action = root.select_action(temperature=tmp)
            state = self.model.get_next_state(state, action)
            reward = self.model.get_reward(state)

            
            if reward is not None:
                ret = []
                for hist_state, hist_action_probs in train_examples:
                    # [Board, actionProbabilities, Reward]
                    ret.append((self.model.state_to_input(hist_state), hist_action_probs, reward))
                # print(hist_state, reward)
                if plotting:
                    self.scores_mcts_dl.append(reward)
                    self.scores_greedy.append(get_greedy_trajectory(self.model))
                    self.scores_random.append(get_random_trajectory(self.model))
                    self.scores_mcts.append(get_MCTS_trajectory(self.model))
                    # plt.plot(self.scores_mcts_dl, label='DeepMCTS')
                    # plt.plot(self.scores_mcts, label='Vanilla MCTS')
                    # plt.plot(self.scores_greedy, label='Greedy policy')
                    # plt.plot(self.scores_random, label='Random policy')
                    # plt.legend()
                    # plt.show()

                return ret

    def learn(self, temperature, random, greedy):
        for i in range(1, self.args['numIters'] + 1):

            print("{}/{}".format(i, self.args['numIters']))

            train_examples = []

            for eps in range(self.args['numEps']):
                iteration_train_examples = self.exceute_episode(temperature, False, random, greedy)
                train_examples.extend(iteration_train_examples)

            self.exceute_episode(0, True, random, greedy)
            shuffle(train_examples)
            self.train(train_examples)
            filename = self.args['checkpoint_path']
            self.save_checkpoint(folder=".", filename=filename)

    def train(self, examples):
        optimizer = optim.Adam(self.dl_model.parameters(), lr=5e-4)
        pi_losses = []
        v_losses = []

        for epoch in range(self.args['epochs']):
          self.dl_model.train()

          batch_idx = 0

          while batch_idx < int(len(examples) / self.args['batch_size']):
            sample_ids = np.random.randint(len(examples), size=self.args['batch_size'])
            boards, pis, vs = list(zip(*[examples[i] for i in sample_ids]))
            boards = torch.FloatTensor(np.array(boards).astype(np.float64))
            target_pis = torch.FloatTensor(np.array(pis))
            target_vs = torch.FloatTensor(np.array(vs).astype(np.float64))

            # predict
            boards = boards.contiguous().cuda()
            target_pis = target_pis.contiguous().cuda()
            target_vs = target_vs.contiguous().cuda()

            # compute output
            out_pi, out_v = self.dl_model(boards)
            l_pi = self.loss_pi(target_pis, out_pi)
            l_v = self.loss_v(target_vs, out_v)
            total_loss = l_pi + l_v

            pi_losses.append(float(l_pi))
            v_losses.append(float(l_v))

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            batch_idx += 1
    

        #   print()
        #   print("Policy Loss", np.mean(pi_losses))
        #   print("Value Loss", np.mean(v_losses))
          # print("Examples:")
          # print(out_pi[0].detach())
          # print(target_pis[0])
          

    def loss_pi(self, targets, outputs):
        loss = -(targets * torch.log(outputs)).sum(dim=1)
        return loss.mean()

    def loss_v(self, targets, outputs):
        loss = torch.sum((targets-outputs.view(-1))**2)/targets.size()[0]
        return loss

    def save_checkpoint(self, folder, filename):
        if not os.path.exists(folder):
            os.mkdir(folder)

        filepath = os.path.join(folder, filename)
        torch.save({
            'state_dict': self.dl_model.state_dict(),
        }, filepath)

In [ ]:
import pandas as pd
import os
import numpy as np
import itertools
import scipy.spatial.distance as distlib
import scipy.stats
import random as rand
import time

class RecommendationSystem:
    def __init__(self, dataset_with_bins, n_features, n_bins):
        '''
        We assume here that the columns of the dataset here are already the binary percentile bins for all features 
        For sanity check: n_features x n_bins = length(dataset_with_bins)
        '''
        self.data = dataset_with_bins # assuming this is the whole dataset split that we want to work with
        self.data.index = np.arange(np.shape(self.data)[0])
        self.n_features = n_features
        self.n_bins = n_bins
        self.k_s = 10 # user picks 10 songs that they enjoy the most
        self.k_t = 10 # for now, just queue the user 10 songs to generate the transition preference model from -- has to be higher than |Action Space| // 2
        self.var_param = 100 # parameter that determines the "variance" of the user reference distribution that the user song picks are generated from
        self.initial_songs = self.gen_user_song_picks() # assuming this is still a pandas df of k_s rows, but only containing the song rows that the user prefers
        
        # Initialize preferences
        self.init_song_preferences()
        self.init_transition_preferences()

    def init_song_preferences(self):
        # Initialize preference array
        self.phi_s = (1/((self.k_s + 1) * self.n_bins)) * np.ones((self.n_features * self.n_bins, 1))
        tmp = (np.sum(self.initial_songs.values, axis = 0)) * (1/(self.k_s + 1))
        self.phi_s = self.phi_s + np.reshape(tmp, (len(tmp), 1))

    def theta_t(self, idx_a, idx_b):
        '''
        Input: indices of songs a and b within the provided dataset (int)

        Output: vector theta_t, assuming the feature sequence of 1-i, 1-2, ..., 1-n_bins, 2-1, ..., n_bins-1, n_bins-2, ..., n_bins-n_bins
        '''
        indices = np.array([], dtype=int)
        for i in range(self.n_features):
            a_bin_idx = np.where(self.data.loc[idx_a][self.data.columns[i*self.n_bins:(i+1)*self.n_bins]] == 1.0)[0]
            b_bin_idx = np.where(self.data.loc[idx_b][self.data.columns[i*self.n_bins:(i+1)*self.n_bins]] == 1.0)[0]
            indices = np.append(indices, int(i*(self.n_bins*self.n_bins) + a_bin_idx*self.n_bins + b_bin_idx))
        out = np.zeros((self.n_bins * self.n_bins * self.n_features, 1))
        out[indices] = 1
        return out

    def init_transition_preferences(self):
        # Initialize user preference vector

        # Take the upper-median preference split
        self.Rs = np.sum(np.matmul(self.data.values, self.phi_s), axis=1)
        self.Mstar = self.data.copy()
        self.Mstar['Rs'] = self.Rs
        self.Mstar.sort_values('Rs', inplace = True, ascending = False)
        self.Mstar = self.Mstar[:np.shape(self.Mstar)[0] // 2]
        self.Mstar['old_index'] = self.Mstar.index
        self.Mstar.index = np.arange(np.shape(self.Mstar)[0])

        # Generate 10th percentile distance of all pairwise distances from M (not M*)
        self.diff = distlib.pdist(self.data.values, 'cosine') #taking cosine distance metric between songs
        self.delta = np.percentile(self.diff, 10, axis=0)
        self.distances = distlib.squareform(self.diff)
        np.fill_diagonal(self.distances, np.inf)

        # Generate a representative subset of M*
        self.representatives = self.delta_medoids(self.Mstar, self.delta)
        if self.k_t > len(self.representatives):
            print("[WARNING] k_t parameter too large for generated representative subset. Consider using a larger dataset or reducing k_t value")
            print("Setting k_t =", len(self.representatives), "to match the generated number of representatives through delta-medoids")
            self.k_t = len(self.representatives)
        song_prev = np.random.choice(self.representatives)
        self.representatives = np.delete(self.representatives, np.argwhere(self.representatives == song_prev))
        #TODO: constraint k_t between length of representatives, give warning message

        self.phi_t = (1/(self.k_t * self.n_bins*self.n_bins)) * np.ones((self.n_features * self.n_bins * self.n_bins, 1))
        
        for i in range(self.k_t-1):
            next_song, theta = self.pick_next_song(song_prev, self.user_ref_samples_t[:, i])
            self.phi_t += (1/(self.k_t)) * theta
            song_prev = next_song

    def pick_next_song(self, previous_song, reference):
        min_dist = 1e06
        for song in self.representatives:
            theta_current = self.theta_t(previous_song, song)
            dist = distlib.cosine(theta_current, reference)
            if dist < min_dist:
                min_theta = theta_current
                min_song = song
                min_dist = dist

        self.representatives = np.delete(self.representatives, np.argwhere(self.representatives == min_song))
        return min_song, min_theta

    def one_shot_delta(self, data, delta, clusters):
        # Remember to change the distance metric in case we change delta distance definition
        distances = distlib.pdist(data[data.columns[:-2]], 'cosine')
        distances = distlib.squareform(distances)
        np.fill_diagonal(distances, np.inf) # Need to populate the diagonals with inf since we look for the smallest off-diagonal value afterwards
        for index, row in data.iterrows():
            dist = 1e6
            representatives = np.array(list(clusters.keys()))
            representatives.dtype = 'int64'

            if len(representatives) > 0:
              # rep = np.where(distances[:, index] == np.min(distances[:, index][representatives]))[0]
              rep = np.intersect1d(np.where(distances[:, index] == np.min(distances[:, index][representatives]))[0], representatives) # Take the intersection to make sure we select the correct index belonging to the set of representatives
              if len(rep) > 1:
                  rep = rep[0]
              else:
                  rep = int(rep)
              dist = distances[rep, index]

            if dist <= delta:
                clusters[rep] = np.append(clusters[rep], index)
            else: 
                rep = index
                clusters[rep] = np.array([rep])

        out = clusters.keys()
        return clusters

    def delta_medoids(self, data, delta):
        distances = distlib.pdist(data[data.columns[:-2]], 'cosine')
        distances = distlib.squareform(distances)
        np.fill_diagonal(distances, 0)
        exit_loop = False
        i = 0
        clusters = {}
        while not exit_loop:
            i +=1
            clusters = self.one_shot_delta(data, delta, clusters)
            if 1 != i:
                representatives_prev = representatives
            else:
                representatives_prev = np.array([], dtype = 'int32')
            representatives = np.array([], dtype = 'int32')
            for cluster in clusters.items():
                cluster = cluster[1]
                cluster_dists = distlib.pdist(data.loc[cluster], 'cosine')
                cluster_dists = distlib.squareform(cluster_dists)
                argmin = np.argmin(np.sum(cluster_dists, axis=0))
                representatives = np.append(representatives, cluster[argmin])
            if np.array_equal(np.sort(representatives), np.sort(representatives_prev)):
                exit_loop = True
        # Convert back to indices of the original data array
        representatives = data['old_index'][np.in1d(data.index, representatives)].values
        return representatives

    def gen_user_song_picks(self):
        """
        Returns a subset of the dataframe of k_s songs closest to the random samples generated from the user preference distribution, simulating picked user reference
        The user reference distribution is generated here and can be accessed through self.user_ref
        """
        self.user_ref_s, self.user_ref_samples_s, self.user_ref_t, self.user_ref_samples_t  = self.gen_user_ref()
        sample_index = np.arange(np.shape(self.data)[0], np.shape(self.data)[0] + self.k_s, dtype='int32')
        to_append = pd.DataFrame(np.transpose(self.user_ref_samples_s), index = sample_index)
        sample = self.data.copy().append(to_append)
        dists_with_samples = distlib.pdist(sample, metric='cosine')
        dists_with_samples = distlib.squareform(dists_with_samples)
        np.fill_diagonal(dists_with_samples, np.inf)
        user_selected_songs = np.array([], dtype='int32')

        for i in sample_index:
            min_ind = np.argmin(dists_with_samples[:sample_index[0], i])
            user_selected_songs = np.append(user_selected_songs, min_ind)
            dists_with_samples[min_ind, :] = np.inf #Do not allow to pick this row anymore
        return self.data.loc[user_selected_songs]
        

    def gen_user_ref(self):
        '''
        Generates a random "ground-truth" user song and transitions preference distributions over features (kind of
        like a ground-truth phi_s and phi_t), as well as k_s for songs (and k_t, for transitions, respectively) samples from it, that are later used to 
        simulate user song preferences and transition preferences that the user preference vectors phi_s and phi_t are learned from

        Input: var_param: a parameter that alters the variance of the distribution over each feature of the ground truth user preference vectors, int

        Output: - user_reference_s (numpy array, n_bins x n_features, 1): ground-truth user song preference distribution over each feature
                - user_samples_s (numpy array, n_bins x n_features, k_s): k_s samples of the "ground truth phi_s", used to select initial, user-preferred songs
                - user_reference_t (numpy array, n_bins*n_bins x n_features, 1): ground-truth user transition preference distribution over each feature
                - user_samples_t (numpy array, n_bins*n_bins x n_features, k_t): k_s samples of the "ground truth phi_t", used to select songs that the user preference phi_t will be estimated from 
        '''
    
        user_reference_s = np.array([])
        user_reference_t = np.array([])

        user_samples_s = np.zeros((self.n_features * self.n_bins, self.k_s))
        user_samples_t = np.zeros((self.n_features * self.n_bins * self.n_bins, self.k_t))

        for i in range(self.n_features):
            vec_s = np.random.randint(1, self.var_param, (self.n_bins, ))
            vec_s = vec_s/np.sum(vec_s)
            user_reference_s = np.append(user_reference_s, vec_s)
            sample_bins_s = np.array(rand.choices(np.arange(self.n_bins, dtype='int32'), vec_s, k = self.k_s))
            user_samples_s[i*self.n_bins + sample_bins_s, np.arange(self.k_s, dtype='int32')] = 1
            
            vec_t = np.random.randint(1, self.var_param, (self.n_bins * self.n_bins, ))
            vec_t = vec_t/np.sum(vec_t)
            user_reference_t = np.append(user_reference_t, vec_t)
            sample_bins_t = np.array(rand.choices(np.arange(self.n_bins * self.n_bins, dtype='int32'), vec_t, k = self.k_t))
            user_samples_t[i*self.n_bins*self.n_bins + sample_bins_t, np.arange(self.k_t, dtype='int32')] = 1

        return user_reference_s, user_samples_s, user_reference_t, user_samples_t


In [ ]:
def ucb_score(parent, child):
    """
    The score for an action that would transition between the parent and child.
    """
    if child.prior < 0.0001:
      return -100000000
    prior_score = child.prior * math.sqrt(parent.visit_count) / (child.visit_count + 1)
    if child.visit_count > 0:
        # The value of the child is from the perspective of the opposing player
        value_score = child.value()
    else:
        value_score = 100000

    return value_score + prior_score

In [3]:
class MDP:
    def __init__(self, file_name, playlist_length):

        df = pd.read_csv(file_name, index_col=None)

        self.features = ['duration', 'key_confidence', 'end_of_fade_in', 'mode_confidence', 'start_of_fade_out', 'tempo',
                    'artist_hotttnesss', 'song_hotttnesss']

        for feature in self.features:
          filter = df[feature] > 0
          df = df[filter]
        
        self.filtered_df = df[self.features]
        self.filtered_df.dropna(inplace=True)
        self.filtered_df = self.filtered_df.head(100)
        self.filtered_df = self.filtered_df.reset_index(drop=True)
        self.n_rows = len(self.filtered_df.index)
        self.n_songs = self.n_rows
        # print(self.n_songs)
        self.n_bins = 10
        self.n_features = len(self.features)
        self.id_song_to_vec = dict()
        self.songs = set()        
        self.song_id_to_song_row = []
        self.song_priors = []
        self.playlist_length = playlist_length
        self.id_song_to_features = dict()

        song_id = 0
      
        for song, row in self.filtered_df.iterrows():
            self.id_song_to_vec[song_id] = np.zeros(80)
            self.id_song_to_features[song_id] = np.zeros(self.n_features)

            for f in range(self.n_features):
              feature = self.features[f]
              self.id_song_to_features[song_id][f] = row[feature]

            self.songs.add(song_id)
            self.song_id_to_song_row.append(row)
            self.song_priors.append(1)
            song_id += 1

        # print(self.id_song_to_features)


        self.songs_list = [i for i in range(self.n_songs)]

        for f in range(self.n_features):

            feature = self.features[f]
            sorted_df = self.filtered_df.sort_values(by=[feature])
            i = 0
            for idx, row in sorted_df.iterrows():
                bin = i * self.n_bins // self.n_rows
                self.id_song_to_vec[idx][f * 10 + bin] = 1
                i += 1


        self.df_input = pd.DataFrame.from_dict(self.id_song_to_vec, orient='index')
        # Select songs that the user prefers (randomly selecting 5 for now, change later)
        rs = RecommendationSystem(self.df_input, self.n_features, self.n_bins)
        self.phi_s = rs.phi_s.reshape(80)
        self.phi_t = rs.phi_t.reshape(800)

        self.transition_reward = np.zeros((self.n_songs, self.n_songs))
        self.song_reward = np.zeros(self.n_songs)
        for i in range(self.n_songs):
          self.song_reward[i] = np.dot(self.phi_s, self.theta_s(i))
          for j in range(self.n_songs):
            self.transition_reward[i,j] = np.dot(self.phi_t, self.get_theta_t(i, j))


    def get_next_state(self, state, action):
      return state + [action]
      
    def get_valid_actions(self, state):
        # All actions are invalid by default
        valid_actions = [0] * self.n_songs

        for song in range(self.n_songs):
            if song not in state:
                valid_actions[song] = 1

        return valid_actions

    def state_to_input(self, state):
      input = np.ones(self.playlist_length*self.n_features*10)*-1
      for i in range(len(state)):
        song = state[i]
        input[i*self.n_features*10:(i+1)*self.n_features*10] = self.id_song_to_vec[song]
      return input

    def get_reward(self, final_state):
      if len(final_state) < self.playlist_length:
        return None
      elif len(final_state) == self.playlist_length:
        state = []
        trajectory_states = [state]
        trajectory_actions = []
        for song in final_state:
          state = self.get_next_state(state, song)
          trajectory_states.append(state)
          trajectory_actions.append(song)
          
        return self.payoff_trajectory(trajectory_states, trajectory_actions)
      else:
        print("Error: length > self.playlist_length")
        return None


    def get_theta_t(self, s1, s2):
        theta_t = np.zeros(800)
        for f in range(len(self.features)):
            for i in range(10):
                for j in range(10):
                    if self.id_song_to_vec[s1][f*10 + i] == 1 and  self.id_song_to_vec[s2][f*10 + j]:
                        theta_t[f*100 + i*10 + j] = 1
        return  theta_t

    def theta_s(self, s):
        return self.id_song_to_vec[s]

    def R(self, s, a):
        Rs = self.song_reward[a]
        Rt = 0
        for i in range(len(s)):
            song_past = s[len(s)-i-1]
            Rt += 1/((i+1)**(0.1)) * self.transition_reward[song_past,a]
        return Rs + Rt


    def payoff_trajectory(self, trajectory_states, trajectory_actions):
      payoff = 0

      for t in range(len(trajectory_actions)):
        payoff +=  self.R(trajectory_states[t], trajectory_actions[t])
      return payoff

    def MC_value(self, s):
      count = 0
      sum_values = 0
      state = s
      # Past episodes
      episode_states = [[]]
      episode_actions = []

      for i in range(model.playlist_length):
        if i < len(s):
          state = s[:i+1]
          action = s[i]
        else:
          action_probs = list(model.song_priors)
          for song_id in range(model.n_songs):
            if song_id in state:
              action_probs[song_id] = 0

          action_probs = np.array(action_probs)/np.sum(np.array(action_probs))
          action = np.random.choice(model.songs_list, 1, p=action_probs)[0]
          state = episode_states[-1] + [action]
        episode_states.append(state)
        episode_actions.append(action)

      # print("MC estimate:", s, episode_states, episode_actions)
          
      # Set MC 
      payoff = self.payoff_trajectory(episode_states, episode_actions)
      
      return payoff

class Node:
    def __init__(self, prior):
        self.visit_count = 0
        self.prior = prior
        self.value_sum = 0
        self.children = {}
        self.state = None

    def expanded(self):
        return len(self.children) > 0

    def value(self):
        if self.visit_count == 0:
            return 0
        return self.value_sum / self.visit_count

    def select_action(self, temperature):
        """
        Select action according to the visit count distribution and the temperature.
        """
        visit_counts = np.array([child.visit_count for child in self.children.values()])
        actions = [action for action in self.children.keys()]
        if temperature == 0:
            action = actions[np.argmax(visit_counts)]
        elif temperature == float("inf"):
            action = np.random.choice(actions)
        else:
            # See paper appendix Data Generation
            visit_count_distribution = visit_counts ** (1 / temperature)
            visit_count_distribution = visit_count_distribution / sum(visit_count_distribution)
            action = np.random.choice(actions, p=visit_count_distribution)

        return action

    def select_child(self):
        """
        Select the child with the highest UCB score.
        """
        best_score = -np.inf
        best_action = -1
        best_child = None

        for action, child in self.children.items():
            score = ucb_score(self, child)
            if score > best_score:
                best_score = score
                best_action = action
                best_child = child

        return best_action, best_child

    def expand(self, state, action_probs):
        """ 
        We expand a node and keep track of the prior policy probability given by neural network
        """
        self.state = state
        for song_id in range(len(action_probs)):
              self.children[song_id] = Node(action_probs[song_id])

    def __repr__(self):
        """
        Debugger pretty print node info
        """
        prior = "{0:.2f}".format(self.prior)
        return "{} Prior: {} Count: {} Value: {}".format(self.state.__str__(), prior, self.visit_count, self.value())


In [4]:
import torch
import math
import numpy as np

class MCTS_DL:

    def __init__(self, dl_model, model, args):
        self.dl_model = dl_model
        self.model = model
        self.args = args

    def run(self, state):
        root = Node(0)

        # EXPAND root
        action_probs, value = self.dl_model.predict(self.model.state_to_input(state))
        valid_actions = self.model.get_valid_actions(state)
        action_probs = action_probs * valid_actions  # mask invalid moves
        action_probs /= np.sum(action_probs)
        root.expand(state, action_probs)

        for _ in range(self.args['num_simulations']):
            node = root
            search_path = [node]

            # SELECT
            while node.expanded():
                action, node = node.select_child()
                search_path.append(node)

            parent = search_path[-2]
            state = parent.state
            # Now we're at a leaf node and we would like to expand
            # Players always play from their own perspective
            next_state = self.model.get_next_state(state, action)
            # Get the board from the perspective of the other player

            # The value of the new state from the perspective of the other player
            value = self.model.get_reward(next_state)
            if value is None:
                # If the game has not ended:
                # EXPAND
                action_probs, value = self.dl_model.predict(self.model.state_to_input(next_state))
                valid_actions = self.model.get_valid_actions(next_state)
                action_probs = action_probs * valid_actions  # mask invalid moves
                action_probs /= np.sum(action_probs)
                node.expand(next_state, action_probs)

            self.backpropagate(search_path, value)

        return root

    def backpropagate(self, search_path, value):
        """
        At the end of a simulation, we propagate the evaluation all the way up the tree
        to the root.l
        """
        for node in reversed(search_path):
            node.value_sum += value 
            node.visit_count += 1

In [5]:
class MCTS:

    def __init__(self, model, args):
        self.model = model
        self.args = args

    def run(self, root_state):
        print("")
        root = Node(0)
        # EXPAND root
        action_probs = list(self.model.song_priors)
        for song_id in range(self.model.n_songs):
          if song_id in root_state:
            action_probs[song_id] = 0
        action_probs /= np.sum(action_probs)
        root.expand(root_state, action_probs)

        for _ in range(self.args['num_simulations']):

            node = root
            search_path = [node]

            # SELECT
            while node.expanded():              
                action, node = node.select_child()
                search_path.append(node)
            parent = search_path[-2]
            state = parent.state
            # # Now we're at a leaf node and we would like to expand
            # # Players always play from their own perspective
            # next_state, _ = self.game.get_next_state(state, action=action)
            # # Get the board from the perspective of the other player
            # next_state = self.game.get_canonical_board(next_state, )

            # The value of the new state from the perspective of the other player
            # value = self.game.get_reward_for_player(next_state)
            next_state = parent.state + [action]
            
            value = model.MC_value(next_state)

            action_probs = list(self.model.song_priors)
            for song_id in range(self.model.n_songs):
              if song_id in parent.state or song_id == action:
                action_probs[song_id] = 0

            node.expand(next_state, action_probs)
        

            # if value is None:
                # If the game has not ended:
                # EXPAND
                # action_probs, value = model.predict(next_state)
                # valid_moves = self.game.get_valid_moves(next_state)
                # action_probs = action_probs * valid_moves  # mask invalid moves
                # action_probs /= np.sum(action_probs)
                # node.expand(next_state, action_probs)

            self.backpropagate(search_path, value)

        return root

    def backpropagate(self, search_path, value):
        """
        At the end of a simulation, we propagate the evaluation all the way up the tree
        to the root.
        """
        for node in reversed(search_path):
            node.value_sum += value
            node.visit_count += 1


In [6]:
def get_random_trajectory(model):
  songs = set(model.songs)
  state = []
  trajectory_states = [state]
  trajectory_actions = []
  for i in range(playlist_length):
    action = random.sample(songs, 1)[0]
    state = model.get_next_state(state, action)
    trajectory_states.append(state)
    trajectory_actions.append(action)
    songs.remove(action)
  cum_rewards = model.payoff_trajectory(trajectory_states, trajectory_actions)
  return cum_rewards

def get_greedy_trajectory(model):
  songs = set(model.songs)
  state = []
  trajectory_states = [state]
  trajectory_actions = []
  for i in range(playlist_length):
    max_score = -np.inf
    max_action = -1
    for try_action in songs:
      try_state = model.get_next_state(state, try_action)
      try_trajectory_states = list(trajectory_states)
      try_trajectory_actions = list(trajectory_actions)
      try_trajectory_states.append(try_state)
      try_trajectory_actions.append(try_action)
      score = model.payoff_trajectory(try_trajectory_states, try_trajectory_actions)
      if score > max_score:
        max_action = try_action
        max_score = score
    
    action = max_action
    state = model.get_next_state(state, action)
    trajectory_states.append(state)
    trajectory_actions.append(action)
    songs.remove(action)
  cum_rewards = model.payoff_trajectory(trajectory_states, trajectory_actions)
  return cum_rewards

def get_MCTS_trajectory(model):
  # MCTS
  mcts = MCTS(model, args)
  trajectory_states = [[]]
  trajectory_actions = []
  state = []
  for i in range(playlist_length):
    root = mcts.run(state)
    counts = [node.visit_count for node in root.children.values()]
    max_val = max(counts)
    song = counts.index(max_val)
    state = list(root.children.values())[song].state
    trajectory_states.append(state)
    trajectory_actions.append(song)
    # print(counts, max_index, state)
    for action, child in root.children.items():
      score = ucb_score(root, child)
      # print(child.state, score)
  cum_rewards =  model.payoff_trajectory(trajectory_states, trajectory_actions)
  return cum_rewards




In [88]:
import torch
import math
import pandas as pd
import numpy as np
import random as rand
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
from datetime import datetime

import scipy.spatial.distance as distlib
playlist_length = 10
model = MDP('MSD.csv', playlist_length)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = {
    'batch_size': 16,
    'numIters': 500,                                # Total number of training iterations
    'num_simulations': 50,                         # Total number of MCTS simulations to run when deciding on a move to play
    'numEps': 50,                                  # Number of full games (episodes) to run during each iteration
    'numItersForTrainExamplesHistory': 20,
    'epochs': 2,                                    # Number of epochs of training per iteration
    'checkpoint_path': 'latest.pth'                 # location to save latest set of weights
}

dl_model = Connect2Model(model.n_features*model.playlist_length*10, model.n_songs, device)
# print(model.n_songs)
trainer = Trainer(dl_model, model, args, [])

# Random
random_scores = [get_random_trajectory(model) for i in range(100)]
avg_random_score = sum(random_scores)/len(random_scores)

greedy_score = get_greedy_trajectory(model)

trainer.learn(1, avg_random_score, greedy_score)
data = {"MCTS": trainer.scores_mcts,
        "MCTS_DL": trainer.scores_mcts_dl,
        "GREEDY": trainer.scores_greedy,
        "RANDOM": trainer.scores_random}

filepath = './data/'

with open(filepath+'{}_{}_{}_{}.pickle'.format(datetime.now().strftime("%y%m%d%H%M%S"), args['numIters'], args['num_simulations'], args['numEps']), 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

/tmp/ipykernel_4941/3035550279.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.filtered_df.dropna(inplace=True)


[WARNING] k_t parameter too large for generated representative subset. Consider using a larger dataset or reducing k_t value
Setting k_t = 7 to match the generated number of representatives through delta-medoids
1/500










2/500










3/500










4/500










5/500










6/500










7/500










8/500










9/500










10/500










11/500










12/500










13/500










14/500










15/500










16/500










17/500










18/500










19/500










20/500










21/500










22/500










23/500










24/500










25/500










26/500










27/500










28/500










29/500










30/500










31/500










32/500










33/500










34/500










35/500










36/500










37/500










38/500










39/500










40/500










41/500










42/500










43/500










44/500










45/500










46/500










47/500










In [27]:
# SHOW THE LIST OF GENERATED FILES
files = os.listdir('./data')
files

['211215003941_5_50_5.pickle',
 '211214235813_100_50_50.pickle',
 'temp_vs_reward.pkl',
 '211215032311_500_50_50.pickle',
 '211215000850_100_50_50.pickle',
 '211215001435_100_50_50.pickle',
 '211215001846_10_50_5.pickle']

In [28]:
# READ THE CORRECT DATA
with open('./data/'+files[3], 'rb') as handle:
    data_in = pickle.load(handle)

In [29]:
sns.set_context('poster')
sns.set_style('whitegrid')

data_in = pd.DataFrame.from_dict(data_in)

In [33]:
%matplotlib qt
data_in.plot()

<AxesSubplot:>